In [1]:
import xarray as xr
import csv
import pandas as pd
import pickle
import numpy as np
import os
from datetime import date, timedelta, datetime
import sys
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import netCDF4 as nc4
import glob
import os.path
import webbrowser
import time
import gsw
import scipy.io as scipy
# from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import matplotlib as mpl
from FUNCTIONS_HYCOM import set_regions, set_regions_tags, find_timedelta64_index, \
create_composite, name_composite, below_above_MLD, plot_simple_pcolor, set_regions_offset, draw_box, \
save_3dvars_as_nc4, save_2dvars_as_nc4, save_1dvars_as_nc4, \
set_cmap, set_clim_plots, set_clim_plots_std, set_clim_plots_diff, set_clim_plots_std_diff, \
function_map_2D_hist, function_plots, function_single_panels_plots
import pickle as pkl
from holteandtalley import HolteAndTalley
import subprocess
import numpy.matlib


In [2]:
# Open netcdf with both HYCOM and forcing data from all events across all regions, at hourly resolution
combined_regions = xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/scripf_for_downloading_forcing_files_CURC/combined_regions_with_rel_hum.nc4')
# combined_regions.depth

In [3]:
# Specify the new depth levels with a step of 5
new_depth_levels = np.arange(0, 510, 5)

# Interpolate along the 'depth' dimension using linear interpolation
combined_regions_interpolated = combined_regions.interp(depth = new_depth_levels, method='linear')


In [4]:
# REMOVE i_time if it's the same as index_time_start 
i_time = int((np.abs(combined_regions.delta_time - int(-2 * 24 * 60 * 60 * 1E9))).argmin()) # time index where to perform the classification (-2 days)

# Needed for composite calculation & plots
index_time_start = int((np.abs(combined_regions.delta_time - int(-2 * 24 * 60 * 60 * 1E9))).argmin()) # 2 days prior
index_time_end = int((np.abs(combined_regions.delta_time - int(14 * 24 * 60 * 60 * 1E9))).argmin()) # 14 days after
index_time_end_10days = int((np.abs(combined_regions.delta_time - int(10 * 24 * 60 * 60 * 1E9))).argmin()) # 14 days after

# Needed for composite plots
offset_lon = 20
timesteps_before = 2*24 # 2 days prior
timesteps_after = 14*24 # 14 days after
time_plot = np.arange(-timesteps_before,timesteps_after+1,1) * 1/24


In [5]:
mask_type_list = ['incr', 'decr']
mode_list = ['fraction', 'fraction', 'fraction', 'fraction', 'absolute', 'absolute', 'absolute', 'absolute', 'absolute', 'absolute', 'absolute']
ML_delta_list = [1, 1/2, 1/3, 2/3, 10, 20, 30, 40, 50, 60, 70]
mask_tag_list = ['1ML', 'halfML', '1thirdML', '2thirdML', 'plus10m', 'plus20m', 'plus30m', 'plus40m', 'plus50m', 'plus60m', 'plus70m']


In [7]:
# Vertical structure of absolute salinity, to use for classification
data = combined_regions_interpolated.salinity_abs

# MLD to use for classification
# MLD = combined_regions_interpolated.salinityMLD_abs
MLD = combined_regions_interpolated.densityMLD_abs

In [8]:
def HYCOM_inc_dec_masks(data, MLD, ML_delta, mode_tag):
    # Initialize lists for masks
    mask_incr = []
    mask_decr = []

    for i_event in np.arange(data.shape[0]):  # loop across events

        # MIXED LAYER
        # Find depth index to insert ML
        i_depth_MLD = np.searchsorted(data[i_event].depth.values, MLD[i_event][i_time])
        # Create new depth axis which includes MLD
        depth_MLD = np.insert(data[i_event].depth.values, i_depth_MLD, MLD[i_event][i_time])
        # Calculate salinity at MLD via interpolation
        salinity_abs_upper = data[i_event][i_time].interp(depth=depth_MLD)

        # Calculate weighted average of salinity within the ML (from surface to MLD)
        # First, need to calculate dz (depth of each layer associated with a salinity value)
        depth_MLD_upper = np.insert(depth_MLD, i_depth_MLD, MLD[i_event][i_time])  # add another MLD value next to the other one
        depth_MLD_upper = np.insert(depth_MLD_upper, 0, 0)  # add another zero at the beginning of the depth

        dz = (depth_MLD_upper[2:i_depth_MLD+3] - depth_MLD_upper[0:i_depth_MLD+1]) / 2  # calculate dz as half of the difference between depth value above and below each depth
        MLS_abs_wgt = (np.sum(dz * salinity_abs_upper[0:i_depth_MLD+1])) / np.sum(dz)  # weighted average from surface to MLD

        # LAYER BELOW ML
        # Calculate bottom limit of layer to consider
        if mode_tag == 'fraction': # e.g., MLD + one third of it
            bottom_limit = MLD[i_event][i_time] + MLD[i_event][i_time] * ML_delta
        elif mode_tag == 'absolute': # e.g., 10 m below MLD
            bottom_limit = MLD[i_event][i_time] + ML_delta   
        # Add another MLD value next to the other one
        depth_below_ML = np.insert(depth_MLD, i_depth_MLD, MLD[i_event][i_time])  # add another MLD value next to the other one
        # Find index of bottom limit
        i_depth_meters_below_MLD = np.searchsorted(depth_below_ML, bottom_limit)
        # Create new depth axis which includes bottom_limit
        depth_below_ML = np.insert(depth_below_ML, i_depth_meters_below_MLD, bottom_limit)  # add bottom_limit
        depth_below_ML = np.insert(depth_below_ML, i_depth_meters_below_MLD, bottom_limit)  # add bottom_limit

        # Calculate salinity at bottom_limit via interpolation
        salinity_abs_lower = data[i_event][i_time].interp(depth=depth_below_ML)

        dz = (depth_below_ML[i_depth_MLD+2:i_depth_meters_below_MLD+2] - depth_below_ML[i_depth_MLD:i_depth_meters_below_MLD])/2  # calculate dz as half of the difference between depth value above and below each depth
        salinity_abs_wgt_lower = (np.sum(dz * salinity_abs_lower[i_depth_MLD+1:i_depth_meters_below_MLD+1])) / np.sum(dz)  # weighted average from MLD to bottom_limit

        # Calculate difference between salinity in layer below ML and weighted average of salinity in the ML
#         mean_salinity_diff_below = np.mean(salinity_abs_wgt_lower - MLS_abs_wgt)
        mean_salinity_diff_below = salinity_abs_wgt_lower - MLS_abs_wgt
#         print(salinity_abs_wgt_lower)
#         print('_________')
#         print(mean_salinity_diff_below)
#         print('_________')
#         print(MLS_abs_wgt)
        # If mean_salinity_diff_below > 0 --> increasing
        if mean_salinity_diff_below > 0: 
            mask_incr.append(1)
            mask_decr.append(0)
        # If mean_salinity_diff_below < 0 --> decreasing
        elif mean_salinity_diff_below < 0: 
            mask_incr.append(0)
            mask_decr.append(1)
        else:  # Nans or other weird stuff
            mask_incr.append(0)
            mask_decr.append(0)
            
    return(mask_incr, mask_decr)


In [9]:
import xarray as xr

# Assuming combined_regions is your xarray.Dataset

folder_name = 'ML_masks'

for imode, iML_delta, imask_tag in zip(mode_list, ML_delta_list, mask_tag_list):
    imask_tuple = HYCOM_inc_dec_masks(data, MLD, iML_delta, imode)
    imask_incr = np.ravel(imask_tuple[0]).astype(bool)
    imask_decr = np.ravel(imask_tuple[1]).astype(bool)

    # Add incremented and decremented masks to the xarray dataset
    combined_regions[f'new_mask_{imode}_{imask_tag}_incr'] = (('index'), imask_incr.reshape((combined_regions.index.size)))
    combined_regions[f'new_mask_{imode}_{imask_tag}_decr'] = (('index'), imask_decr.reshape((combined_regions.index.size)))

    # Save the masks
    filename_incr = f"{folder_name}/mask_{imode}_{imask_tag}_incr.pkl"
    filename_decr = f"{folder_name}/mask_{imode}_{imask_tag}_decr.pkl"

    with open(filename_incr, 'wb') as file:
        pickle.dump(imask_incr, file)

    with open(filename_decr, 'wb') as file:
        pickle.dump(imask_decr, file)

    print(f"Saved {filename_incr} and {filename_decr}")

combined_regions.to_netcdf('combined_regions_with_new_masks_densityMLD.nc4')
    

Saved ML_masks/mask_fraction_1ML_incr.pkl and ML_masks/mask_fraction_1ML_decr.pkl
Saved ML_masks/mask_fraction_halfML_incr.pkl and ML_masks/mask_fraction_halfML_decr.pkl
Saved ML_masks/mask_fraction_1thirdML_incr.pkl and ML_masks/mask_fraction_1thirdML_decr.pkl
Saved ML_masks/mask_fraction_2thirdML_incr.pkl and ML_masks/mask_fraction_2thirdML_decr.pkl
Saved ML_masks/mask_absolute_plus10m_incr.pkl and ML_masks/mask_absolute_plus10m_decr.pkl
Saved ML_masks/mask_absolute_plus20m_incr.pkl and ML_masks/mask_absolute_plus20m_decr.pkl
Saved ML_masks/mask_absolute_plus30m_incr.pkl and ML_masks/mask_absolute_plus30m_decr.pkl
Saved ML_masks/mask_absolute_plus40m_incr.pkl and ML_masks/mask_absolute_plus40m_decr.pkl
Saved ML_masks/mask_absolute_plus50m_incr.pkl and ML_masks/mask_absolute_plus50m_decr.pkl
Saved ML_masks/mask_absolute_plus60m_incr.pkl and ML_masks/mask_absolute_plus60m_decr.pkl
Saved ML_masks/mask_absolute_plus70m_incr.pkl and ML_masks/mask_absolute_plus70m_decr.pkl


In [10]:
xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/combined_regions_with_new_masks_salinityMLD.nc4')


<xarray.Dataset>
Dimensions:                          (index: 1238, delta_time: 721, depth: 28)
Coordinates:
  * index                            (index) int64 0 1 2 3 ... 1235 1236 1237
  * depth                            (depth) float64 0.0 2.0 4.0 ... 400.0 500.0
    longitude                        (index) float64 ...
    latitude                         (index) float64 ...
    integral_precip                  (index) float64 ...
    sel_date                         (index) object ...
  * delta_time                       (delta_time) timedelta64[ns] -6 days +00...
Data variables: (12/57)
    temperature_abs                  (index, delta_time, depth) float64 ...
    salinity_abs                     (index, delta_time, depth) float64 ...
    density_abs                      (index, delta_time, depth) float64 ...
    temperature_anom                 (index, delta_time, depth) float64 ...
    salinity_anom                    (index, delta_time, depth) float64 ...
    density_anom                     (index, delta_time, depth) float64 ...
    ...                               ...
    new_mask_absolute_plus50m_incr   (index) bool ...
    new_mask_absolute_plus50m_decr   (index) bool ...
    new_mask_absolute_plus60m_incr   (index) bool ...
    new_mask_absolute_plus60m_decr   (index) bool ...
    new_mask_absolute_plus70m_incr   (index) bool ...
    new_mask_absolute_plus70m_decr   (index) bool ...
Attributes:
    description:  HYCOM T/S co-located with TCs

In [11]:
xr.open_dataset('/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/combined_regions_with_new_masks_densityMLD.nc4')


<xarray.Dataset>
Dimensions:                          (index: 1238, delta_time: 721, depth: 28)
Coordinates:
  * index                            (index) int64 0 1 2 3 ... 1235 1236 1237
  * depth                            (depth) float64 0.0 2.0 4.0 ... 400.0 500.0
    longitude                        (index) float64 ...
    latitude                         (index) float64 ...
    integral_precip                  (index) float64 ...
    sel_date                         (index) object ...
  * delta_time                       (delta_time) timedelta64[ns] -6 days +00...
Data variables: (12/57)
    temperature_abs                  (index, delta_time, depth) float64 ...
    salinity_abs                     (index, delta_time, depth) float64 ...
    density_abs                      (index, delta_time, depth) float64 ...
    temperature_anom                 (index, delta_time, depth) float64 ...
    salinity_anom                    (index, delta_time, depth) float64 ...
    density_anom                     (index, delta_time, depth) float64 ...
    ...                               ...
    new_mask_absolute_plus50m_incr   (index) bool ...
    new_mask_absolute_plus50m_decr   (index) bool ...
    new_mask_absolute_plus60m_incr   (index) bool ...
    new_mask_absolute_plus60m_decr   (index) bool ...
    new_mask_absolute_plus70m_incr   (index) bool ...
    new_mask_absolute_plus70m_decr   (index) bool ...
Attributes:
    description:  HYCOM T/S co-located with TCs

# Compare the True elements of each loaded mask with the True elements of the plus50m mask. You want to print how many times these True elements match and the percentage of agreement. 

In [ ]:
### INCREASING
import os
import pickle
import numpy as np

# Reload the "plus50m" mask
plus50m_mask_file = "/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_absolute_plus50m_incr.pkl"
with open(plus50m_mask_file, 'rb') as file:
    plus50m_mask_incr = pickle.load(file)

# Create a dictionary to store the loaded masks
loaded_masks = {}

for imode, iML_delta, imask_tag in zip(mode_list, ML_delta_list, mask_tag_list):
    for suffix in ['incr']:
        # Load each mask
        filename = f"/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_{imode}_{imask_tag}_{suffix}.pkl"
        filename_base = os.path.basename(filename)

        with open(filename, 'rb') as file:
            loaded_mask = pickle.load(file)

        # Find the indices of True elements in both masks
        plus50m_true_indices = np.where(plus50m_mask_incr)
        loaded_mask_true_indices = np.where(loaded_mask)

        # Find the common True elements between the masks
        common_true_indices = np.intersect1d(plus50m_true_indices, loaded_mask_true_indices)

        # Calculate the number and percentage of common True elements
        num_common_true_elements = len(common_true_indices)
        percentage_common_true = (num_common_true_elements / len(plus50m_true_indices[0])) * 100

        print(f"Mask {filename_base} has {num_common_true_elements} common True elements with plus50m mask")
        print(f"Percentage of common True elements: {percentage_common_true:.2f}%")
        print('=====================================================================')


In [ ]:
### DECREASING
import os
import pickle
import numpy as np

# Reload the "plus50m" mask
plus50m_mask_file = "/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_absolute_plus50m_decr.pkl"
with open(plus50m_mask_file, 'rb') as file:
    plus50m_mask_decr = pickle.load(file)

# Create a dictionary to store the loaded masks
loaded_masks = {}

for imode, iML_delta, imask_tag in zip(mode_list, ML_delta_list, mask_tag_list):
    for suffix in ['decr']:
        # Load each mask
        filename = f"/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_{imode}_{imask_tag}_{suffix}.pkl"
        filename_base = os.path.basename(filename)

        with open(filename, 'rb') as file:
            loaded_mask = pickle.load(file)

        # Find the indices of True elements in both masks
        plus50m_true_indices = np.where(plus50m_mask_decr)
        loaded_mask_true_indices = np.where(loaded_mask)

        # Find the common True elements between the masks
        common_true_indices = np.intersect1d(plus50m_true_indices, loaded_mask_true_indices)

        # Calculate the number and percentage of common True elements
        num_common_true_elements = len(common_true_indices)
        percentage_common_true = (num_common_true_elements / len(plus50m_true_indices[0])) * 100

        print(f"Mask {filename_base} has {num_common_true_elements} common True elements with plus50m mask")
        print(f"Percentage of common True elements: {percentage_common_true:.2f}%")
        print('=====================================================================')


# Also, we should check how many profiles are increasing (or decreasing) in ALL the masks and compare that number to the number of increasing (or decreasing) profiles using the 50m mask. 

In [ ]:
import os
import pickle
import numpy as np

# Reload the "plus50m" masks
plus50m_incr_mask_file = "/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_absolute_plus50m_incr.pkl"
plus50m_decr_mask_file = "/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_absolute_plus50m_decr.pkl"

with open(plus50m_incr_mask_file, 'rb') as file:
    plus50m_incr_mask = pickle.load(file)

with open(plus50m_decr_mask_file, 'rb') as file:
    plus50m_decr_mask = pickle.load(file)

# Create a dictionary to store the loaded masks
loaded_masks = {}

for imode, iML_delta, imask_tag in zip(mode_list, ML_delta_list, mask_tag_list):
    for suffix in ['decr', 'incr']:
        # Load each mask
        filename = f"/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_{imode}_{imask_tag}_{suffix}.pkl"
        filename_base = os.path.basename(filename)

        with open(filename, 'rb') as file:
            loaded_mask = pickle.load(file)

        # Compare with the plus50m increasing mask
        if suffix == 'incr':
            common_increasing = np.sum(loaded_mask & plus50m_incr_mask)
            perc_common = common_increasing/num_true_increasing_plus50m*100
            print(f"Out of Number of  {num_true_increasing_plus50m} increasing profiles in the plus50m mask, {common_increasing} are also in the {filename_base} mask ({perc_common}%)")


        # Compare with the plus50m decreasing mask
        # Compare with the plus50m increasing mask
        if suffix == 'decr':
            common_decreasing = np.sum(loaded_mask & plus50m_decr_mask)
            perc_common = common_decreasing/num_true_decreasing_plus50m*100
            print(f"Out of Number of  {num_true_decreasing_plus50m} decreasing profiles in the plus50m mask, {common_decreasing} are also in the {filename_base} mask ({perc_common}%)")


        print('=====================================================================')
        


In [ ]:
# # Increasing VS Decreasing - old method - WRONG

# # Reload the "plus50m" mask
# plus50m_mask_file = "/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_absolute_plus50m_decr.pkl"
# with open(plus50m_mask_file, 'rb') as file:
#     plus50m_mask_decr = pickle.load(file)

# # Create a dictionary to store the loaded masks
# loaded_masks = {}

# for imode, iML_delta, imask_tag in zip(mode_list, ML_delta_list, mask_tag_list):
#     for suffix in ['decr']:
#         # Load each mask
#         filename = f"/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_{imode}_{imask_tag}_{suffix}.pkl"
#         filename_base = os.path.basename(filename)

#         with open(filename, 'rb') as file:
#             loaded_mask = pickle.load(file)

#         # Compare with the "plus50m" mask
#         is_equal = np.array_equal(plus50m_mask_decr, loaded_mask)
#         print(f"Mask {filename_base} is {'equal' if is_equal else 'not equal'} to the plus50m mask")

#         # Find the number of equal elements
#         num_equal_elements = np.sum(plus50m_mask_decr == loaded_mask)
#         total_elements = plus50m_mask_decr.size
#         percentage_equal = (num_equal_elements / total_elements) * 100
#         percentage_NOT_equal = 100 - percentage_equal

#         print(f"Number of equal elements: {num_equal_elements}")
#         print(f"Percentage of equal elements: {percentage_equal:.2f}%")
#         print(f"Number of NOT equal elements: {total_elements - num_equal_elements}")
#         print(f"Percentage of NOT equal elements: {percentage_NOT_equal:.2f}%")

#         print('=====================================================================')

        
# # Reload the "plus50m" mask
# plus50m_mask_file = "/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_absolute_plus50m_incr.pkl"
# with open(plus50m_mask_file, 'rb') as file:
#     plus50m_mask_incr = pickle.load(file)

# # Create a dictionary to store the loaded masks
# loaded_masks = {}

# for imode,iML_delta,imask_tag in zip(mode_list, ML_delta_list, mask_tag_list):

#     for suffix in ['incr']:
#         # Load each mask
#         filename = f"/Users/jacoposala/Desktop/CU/3.RESEARCH/ARGO_analysis/TC_HYCOM/ML_masks/mask_{imode}_{imask_tag}_{suffix}.pkl"
#         filename_base = os.path.basename(filename)

#         with open(filename, 'rb') as file:
#             loaded_mask = pickle.load(file)
        
#         # Compare with the "plus50m" mask
#         is_equal = np.array_equal(plus50m_mask_incr, loaded_mask)
#         print(f"Mask {filename_base} is {'equal' if is_equal else 'not equal'} to the plus50m mask")

#         # Find the number of equal elements
#         num_equal_elements = np.sum(plus50m_mask_incr == loaded_mask)
#         total_elements = plus50m_mask_incr.size
#         percentage_equal = (num_equal_elements / total_elements) * 100
#         percentage_NOT_equal = 100 - percentage_equal

#         print(f"Number of equal elements: {num_equal_elements}")
#         print(f"Percentage of equal elements: {percentage_equal:.2f}%")
#         print(f"Number of NOT equal elements: {total_elements - num_equal_elements}")
#         print(f"Percentage of NOT equal elements: {percentage_NOT_equal:.2f}%")
        
#         print('=====================================================================')
